In [16]:
from pathlib import Path
from astropy.io import fits 
import os
import astropy
from fast_histogram import histogram, histogram2d
from matplotlib import animation
from IPython.display import HTML # requires ffmpeg
from astropy.wcs import WCS
import astropy.time as atime
import datetime

skypos = (352.9378,-2.7447) # position of AF Psc

In [2]:
swift_path = '/Users/katborski/Documents/GitHub/AFPSC/Swift/'
swift_path = '/Volumes/MERTOOLS/AFPSC/Swift/'
#os.path.isdir(swift_path)

In [3]:
event_files = []
for visit in os.listdir(swift_path):
    if visit == '00087201006': # Does not have event data
        continue
    if visit == '00014769003': # Does not have data
        continue
for fn in os.listdir(swift_path):
    if 'DS_Store' in fn:      #skip over 
        continue
    visit_path = swift_path+visit+'/'
    for img in os.listdir(visit_path+'uvot/event/'):
        fn=visit_path+'uvot/event/'+img
        data = fits.open(fn)
        event_files+=[fn]

print(len(event_files))

29


In [4]:
data[0].header


SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                    8 / number of bits per data pixel                  
NAXIS   =                    0 / number of data axes                            
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
TELESCOP= 'SWIFT '             / Telescope (mission) name                       
INSTRUME= 'UVOTA '             / Instrument name                                
DETNAM  = '1400    '                                                            
FILTER  = 'U'                  / Filter                                         
OBS_ID  = '00014769020'        / Observation ID                                 
TARG_ID =                14769 / Target ID                                      
SEG_NUM =                   

In [5]:
data.info()

Filename: /Volumes/MERTOOLS/AFPSC/Swift/00014769020/uvot/event/sw00014769020uuuw1po_uf.evt.gz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     108   ()      
  1  EVENTS        1 BinTableHDU    324   20189881R x 9C   [1D, 1I, 1I, 1I, 1I, 1I, 1I, 1I, 1I]   
  2  STDGTI        1 BinTableHDU     58   2R x 2C   [1D, 1D]   
  3  GTI1          1 BinTableHDU     60   2R x 2C   [1D, 1D]   
  4  WINDOW        1 BinTableHDU    144   1R x 18C   [1D, 1J, 1I, 1I, 1J, 1J, 1J, 1J, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D]   


In [6]:
print(data[0].header['TSTART'])
print(data[0].header['TSTOP'])

652446378.54316
652448040.887398


In [28]:
UTC_start = data[0].header['MJDREFI']  + (data[0].header['TSTART'] + data[0].header['UTCFINIT'])/86400.0 #(in days)
print(UTC_start, "including reference date") #Start time in Days including 
days_sinceStart = UTC_start-51910
year_start = (UTC_start - data[0].header['MJDREFI'])/365
print(year_start, "years since reference date")
#Initial Reference Date Jan 1, 2001 (51910)
print(UTC_start)

59461.46239438727 including reference date
20.688938066814444 years since reference date
59461.46239438727


In [8]:
UTC_stop = data[0].header['MJDREFI']  + (data[0].header['TSTOP'] + data[0].header['UTCFINIT'])/86400.0 #(in days)
print(UTC_stop, "including reference date")
days_sinceEnd = UTC_stop-51910
#same date as above
year_stop = (UTC_stop - data[0].header['MJDREFI'])/365
print(year_stop, "years since reference date")


59461.48163448262 including reference date
20.68899077940443 years since reference date


In [9]:
timestamp_start = datetime.datetime(2001,1,1) + datetime.timedelta(days_sinceStart)
print(timestamp_start)

2021-09-04 11:05:50.875060


In [10]:
timestamp_end = datetime.datetime(2001,1,1) + datetime.timedelta(days_sinceEnd)
print(timestamp_end)

2021-09-04 11:33:33.219298


# new stuff

In [20]:
epoch = atime.Time('2001-01-01 00:00:00')

In [27]:
dt_start = atime.TimeDelta(data[0].header['TSTART'],format='sec')
print((epoch+dt_start).iso)

2021-09-04 11:06:13.543


In [34]:
dt_start = atime.TimeDelta(data[0].header['TSTART'] + data[0].header['UTCFINIT'],format='sec')
print((epoch+dt_start).mjd)

59461.462336516895


In [48]:
def swift_time_to_utc(time,epoch = atime.Time('2001-01-01 00:00:00')):
    dt_start = atime.TimeDelta(time,format='sec')
    return (epoch+dt_start).mjd

In [50]:
t = swift_time_to_utc(data[1].data['TIME'])